In [2]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.0 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-huggingface


In [7]:
from PyPDF2 import PdfFileReader
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import Together

def setup_rag_model(pdf_path, together_api_key):
    # Load the PDF document
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Extract text from each page and combine them into a single document
    document_text = " ".join([page.page_content for page in pages])

    # Create embeddings
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(pages, embedding_function)

    # Initialize the LLM
    llm = Together(
        model="meta-llama/Llama-2-70b-chat-hf",
        max_tokens=256,
        temperature=0,
        top_k=1,
        together_api_key=together_api_key
    )

    # Setup the retriever
    retriever = db.as_retriever(similarity_score_threshold=0.9)

    # Define the prompt template for Q&A
    prompt_template = """Please answer the following question based on the document provided:
    CONTEXT: {context}
    QUESTION: {question}"""
    PROMPT = PromptTemplate(template=f"[INST] {prompt_template} [/INST]", input_variables=["context", "question"])

    # Create the retrieval chain
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        input_key='query',
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT},
        verbose=True
    )

    return chain, document_text

def ask_question(chain, document_text, question):
    response = chain({"query": question, "context": document_text})
    return response['result']

# Example usage
pdf_path = "/content/somatosensory 1.pdf"  # Replace with the correct path to your PDF document
together_api_key = "5c3aaa83cb623ccf29c3038870387e97d17f64112b00d431f55f6fc19e4d75ab"  # Your Together API key

# Validate the API key
if not together_api_key or together_api_key == "your_together_api_key":
    raise ValueError("Invalid Together API key provided. Please ensure you have a valid key.")

# Setup the RAG model
rag_chain, document_text = setup_rag_model(pdf_path, together_api_key)

# Example Questions
questions = [
    "What is the main topic of this document?",
    "What are the key points discussed?",
    "Can you provide details on the specific topic mentioned?",
]

# Get answers to the questions
for question in questions:
    answer = ask_question(rag_chain, document_text, question)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")




> Entering new RetrievalQA chain...

> Finished chain.
Question: What is the main topic of this document?
Answer:   The main topic of this document is the anatomy of the somatosensory system, which includes sensors in the skin and sensors in muscles, tendons, and joints that provide information about temperature, pressure, surface texture, pain, muscle length, muscle tension, and joint angles.



> Entering new RetrievalQA chain...

> Finished chain.
Question: What are the key points discussed?
Answer:   The key points discussed in the document include:

1. The somatosensory system consists of sensors in the skin and sensors in muscles, tendons, and joints.
2. Cutaneous receptors in the skin tell us about temperature, pressure, surface texture, and pain.
3. Rapidly adapting afferents from Meissner corpuscles and hair receptors at the roots of hairs respond to small displacements of objects during the early stages of lifting and lead to adjustments in grip force.
4. Encapsulated recep